In [42]:
import pandas as pd
from ast import literal_eval

In [3]:
data = pd.read_csv('processed_1.csv')
data.iloc[10]

r_id                                                       4.3
doc_id                                                       4
specialty                                                   -1
gender                                                       f
S                                                            5
P                                                            5
H                                                            5
K                                                            5
review       ['working', 'quite', 'time', 'say', 'enough', ...
Name: 10, dtype: object

In [4]:
df = data.groupby('doc_id').apply(lambda x: x.sum())

In [5]:
df.iloc[1]

r_id                                                      12.6
doc_id                                                      12
specialty                                 psychpsychpsychpsych
gender                                                    ffff
S                                                           12
P                                                           13
H                                                           12
K                                                           12
review       ['really', 'enjoy', 'visit', 'pleasant', 'talk...
Name: 3, dtype: object

In [7]:
# make a set of specialties and genders
specialties = set(data['specialty'])
genders = set(data['gender'])
print(genders)
print(specialties)

{'m', 'f'}
{'-1', 'family', 'surg', 'psych', 'skin', 'pedia', 'brain', 'gyn', 'int', 'bone', 'eye', 'gas', 'endo', 'ent'}


In [13]:
d = data[['doc_id','specialty','gender']].set_index('doc_id').T.to_dict('list')
d

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  


In [18]:
specialties = list(specialties)
specialties.remove('-1')
specialties.append('-1') # Now we've got -1 at the end, as default.
s_map = {}
for i,s in enumerate(specialties):
    s_map[s] = i
genders = list(genders)
print(len(specialties),len(genders))
genders

14 2


['m', 'f']

In [19]:
specialties

['family',
 'surg',
 'psych',
 'skin',
 'pedia',
 'brain',
 'gyn',
 'int',
 'bone',
 'eye',
 'gas',
 'endo',
 'ent',
 '-1']

In [61]:
for i,row in df.iterrows():
    try:
        name = row.name
        temp = row['review'].replace("][", ",")
        temp = literal_eval(temp)
        df.set_value(name,'review', temp)
    except:
        print("####",row['review'],'\n')
        continue

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  


In [99]:
# vocab
V_freq = {}
    
for i,row in df.iterrows():
    for w in row['review']:
        try:
            V_freq[w] += 1
        except:
            V_freq[w] = 1
import operator
sorted_V_freq = sorted(V_freq.items(), key=operator.itemgetter(1))

In [100]:
vocab_size = 1000 # 500

V_map = {}
V_new = sorted_V_freq[-vocab_size:]
print(V_new)
for i,tup in enumerate(V_new):
    V_map[tup[0]] = i
print(len(V_map))
list(V_map.keys())[:10]

[('timely', 156), ('encountered', 156), ('cell', 157), ("doctor's", 157), ('educated', 157), ('allowed', 158), ('worried', 159), ('receiving', 159), ('class', 159), ('terrific', 159), ('causing', 160), ('unlike', 160), ('struggling', 160), ('missed', 160), ('conceive', 160), ('podiatrist', 160), ('heal', 160), ('shot', 161), ('easily', 161), ('avoid', 162), ('organized', 162), ('2016', 162), ('demeanor', 163), ('calming', 163), ('manager', 164), ('healed', 164), ('sit', 164), ('specific', 164), ('holistic', 164), ('lack', 164), ('fabulous', 165), ('cold', 165), ('tear', 165), ('latest', 165), ('non', 165), ('travel', 165), ('refused', 165), ('mention', 166), ('expensive', 166), ('action', 167), ('wear', 167), ('involved', 167), ('comfort', 167), ('ob', 168), ('ultrasound', 168), ('list', 168), ('mark', 169), ('scan', 169), ('lifestyle', 170), ('effort', 170), ('par', 170), ('empathy', 170), ('learned', 171), ('push', 171), ('understands', 172), ('sensitive', 172), ('temp', 172), ('asks

['timely',
 'encountered',
 'cell',
 "doctor's",
 'educated',
 'allowed',
 'worried',
 'receiving',
 'class',
 'terrific']

In [101]:
S = len(specialties)
V = len(V_map)
#size_of_var = 2+2*V+2*S+2*S*V
size_of_var = 2+V+2*S+2*S*V # beta_w no longer needed.
print(1,2,2+S,2+S+V,2+S+V+S,2+S+V+S+S*V,size_of_var)

1 2 16 1016 1030 15030 29030


In [102]:
y = {}
for s in specialties:
    y[s] = {'m':{},'f':{}}

# dict to store doctor wise frequencies.
V_beta_w = {}
    
# make set out of text aggregations
for i in range(len(df)):
    words = set()
    entry = df.iloc[i]
    if d[entry.name][0] == '-1':
        continue
    for w in entry['review']:
        words.add(w)
# and fill the y 3d matrix (y_sgw)
    sg = d[entry.name]
    for w in words:
        if w not in V_map.keys():
            continue
        try:
            V_beta_w[w] += 1
        except:
            V_beta_w[w] = 1
        try:
            y[sg[0]][sg[1]][w] += 1
        except:
            y[sg[0]][sg[1]][w] = 1

In [103]:
type(y['int']['m'])
len(V_beta_w.keys())

1000

In [104]:
def assign(s,g,w):
    var = bitarray(size_of_var)
    var.setall(0)
    
    var[0] = 1
    var[2+s_map[s]] = 1
    if g == 'f':
        var[1] = 1 # beta_g
        var[2+S+V_map[w]] = 1 # beta_gw
        var[2+S+V+s_map[s]] = 1 # beta_gs
        var[2+S+V+S+S*V+ (s_map[s]*V) + (V_map[w])] = 1 # beta_sgw
    var[2+S+V+S+ (s_map[s]*V) + (V_map[w]) ] = 1 # beta_sw
    return var

In [105]:
from bitarray import bitarray

In [106]:
import numpy as np

size_of_var1 = 1+1+V+S
def assign2(s,g,w):
    var = bitarray(size_of_var1)
    var.setall(0)
    var[0] = 1
    var[2+V_map[w]] = 1
    var[2+V+s_map[s]] = 1
    if g == 'f':
        var[1] = 1
    return var

In [107]:
import math
count = 0
for s in specialties:
    for g in genders:
        for w in list(y[s][g].keys()):
            count+=1
            # take log, discount the log of general word frequency, i.e. no need to train beta_w.
            y[s][g][w] = math.log(y[s][g][w]) - math.log(V_beta_w[w])
count   

23608

In [108]:
y['gyn']['f'].keys()

dict_keys(['finally', 'easy', 'worked', 'addressed', 'made', 'email', 'husband', 'bedside', 'u', 'fact', 'pregnant', 'despite', 'she', 'work', 'year', 'grateful', 'everything', 'couple', 'schedule', 'address', 'believe', 'hard', 'getting', 'reassuring', 'amazing', 'miracle', 'even', 'come', 'manner', 'after', 'sure', 'could', 'answered', 'fear', 'not', 'important', 'accommodating', 'her', 'sent', 'extremely', 'probably', 'thorough', 'wonderful', 'hand', 'one', 'job', 'him', 'attitude', 'day', 'rushed', 'like', 'love', 'assistant', 'fantastic', 'ever', 'never', 'feel', 'great', 'last', 'attentive', 'appointment', 'tell', 'insurance', 'look', 'punctual', 'issue', 'helped', 'however', 'really', 'friend', 'excellent', 'knowledgeable', 'fertility', 'go', 'take', 'anyone', 'missed', 'specialist', 'caring', 'kind', 'give', 'office', 'intelligent', 'highly', 'awesome', 'going', 'delivered', 'personable', 'recommend', 'beyond', 'thank', 'available', 'mine', 'phone', 'course', 'later', 'woman', 

In [109]:
y['gyn']['f']['always']

-2.630473585923867

In [110]:
count = 0
X,Y = [],[]
for s in specialties:
    for g in genders:
        for w in list(y[s][g].keys()):
            x = assign(s,g,w)
            ysgw = y[s][g][w]
            X.append(x)
            Y.append(ysgw)
            count+=1
            if count%100 == 0:
                print(count)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

In [111]:
import pickle
LR = {'X':X,'Y':Y}
version = 'july23'
with open('LR_'+version+'.pickle','wb') as h:
    pickle.dump(LR,h,protocol=2)

In [112]:
#import pickle
#version = 'june29_3'
#with open('LR_'+version+'.pickle','rb') as h:
#    LR = pickle.load(h)

In [113]:
X = LR['X']
Y = LR['Y']
Y[500]

-3.51650822817315

In [114]:
S = len(specialties) # 13
V = vocab_size # 1000
print(len(X)) # 11612
2+S+V+S+V*S+((S)*V)-1 # 12525
print(len(X[0])) # 12526

23608
29030


In [141]:
# overcounting of variables to be addressed:
ig = [] # the indices to be discounted.
ig.append(2+S-1)
ig.append(2+S+V-1)
#ig.append(2+S+V+S-1) counted in the i=0 next loop.
for i in range(S):
    ig.append(2+S+V+S+(i*V)-1)
# 2+S+V+S+((S-1)*V)-1 counted.

# If you don't want beta_sgw:
for i in range(2+S+V+S+(S-1)*V-1,len(X[0])):
    ig.append(i)

In [143]:
ig # ignored / discounted ones (which had earlier been overcounted)

[15,
 1015,
 1029,
 2029,
 3029,
 4029,
 5029,
 6029,
 7029,
 8029,
 9029,
 10029,
 11029,
 12029,
 13029,
 14029,
 14029,
 14030,
 14031,
 14032,
 14033,
 14034,
 14035,
 14036,
 14037,
 14038,
 14039,
 14040,
 14041,
 14042,
 14043,
 14044,
 14045,
 14046,
 14047,
 14048,
 14049,
 14050,
 14051,
 14052,
 14053,
 14054,
 14055,
 14056,
 14057,
 14058,
 14059,
 14060,
 14061,
 14062,
 14063,
 14064,
 14065,
 14066,
 14067,
 14068,
 14069,
 14070,
 14071,
 14072,
 14073,
 14074,
 14075,
 14076,
 14077,
 14078,
 14079,
 14080,
 14081,
 14082,
 14083,
 14084,
 14085,
 14086,
 14087,
 14088,
 14089,
 14090,
 14091,
 14092,
 14093,
 14094,
 14095,
 14096,
 14097,
 14098,
 14099,
 14100,
 14101,
 14102,
 14103,
 14104,
 14105,
 14106,
 14107,
 14108,
 14109,
 14110,
 14111,
 14112,
 14113,
 14114,
 14115,
 14116,
 14117,
 14118,
 14119,
 14120,
 14121,
 14122,
 14123,
 14124,
 14125,
 14126,
 14127,
 14128,
 14129,
 14130,
 14131,
 14132,
 14133,
 14134,
 14135,
 14136,
 14137,
 14138,
 1413

In [117]:
print(len(ig)) # 1025
ig = list(set(ig))
len(ig)

15017


15016

In [118]:
import numpy as np
%time X1 = np.array(X)
X1.shape

CPU times: user 21.9 s, sys: 677 ms, total: 22.6 s
Wall time: 22.6 s


(23608, 29030)

In [119]:
%time X2 = np.delete(X1,ig,axis=1)
X2.shape

CPU times: user 2.56 s, sys: 264 ms, total: 2.83 s
Wall time: 2.91 s


(23608, 14014)

In [121]:
from sklearn import linear_model
regr = linear_model.Ridge()

In [122]:
%time regr.fit(X2,Y)

CPU times: user 2min 13s, sys: 11.4 s, total: 2min 24s
Wall time: 1min 26s


Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [127]:
c = regr.coef_
regr.intercept_

-3.2717542600990317

In [128]:
c[1] # FEMALE

-0.4923888186505001

In [129]:
c[2:2+S] # SPECIALTIES

array([-0.2966371 , -0.14148597, -0.1643354 ,  0.69751674, -0.53423729,
        0.65324227,  0.81079624, -0.33705595,  0.51157672, -0.04790005,
       -0.60668993, -0.51629807, -0.02849221, -0.02142843])

In [130]:
beta_w = {}
for i,w in enumerate(c[2:2+V]):
    beta_w[V_new[i]] = w
sorted_beta_w = sorted(beta_w.items(), key=operator.itemgetter(1))
sorted_beta_w[0:10]

[(('positive', 1069), -2.1224383890846985),
 (('information', 594), -1.4375564000727379),
 (('professionalism', 507), -1.0532583256029866),
 (('provides', 317), -0.9987279930500776),
 (('causing', 160), -0.6066899323329684),
 (('hear', 238), -0.5851894007534842),
 (('used', 469), -0.5814124816658479),
 (('educated', 157), -0.5342372892430469),
 (('away', 954), -0.5232005941542265),
 (('thoroughly', 358), -0.5178555940931054)]

In [131]:
beta_gw = {}
for i,w in enumerate(c[2+S-1:2+S+V-2]):
    beta_gw[V_new[i]] = w
sorted_beta_gw = sorted(beta_gw.items(), key=operator.itemgetter(1))
len(c[2+S-1:2+S+V-2])

999

In [132]:
list(beta_gw.keys())[0]

('timely', 156)

In [134]:
sorted_beta_gw[0:10]

[(("he's", 969), -2.1224383890846985),
 (('him', 9242), -1.7120516157937204),
 (('his', 14264), -1.5598187750644055),
 (('man', 573), -1.4375564000727379),
 (('guy', 490), -1.0532583256029866),
 (('himself', 304), -0.9987279930500776),
 (('2017', 234), -0.5851894007534842),
 (('healing', 445), -0.5814124816658479),
 (('recovery', 886), -0.5232005941542265),
 (('successful', 348), -0.5178555940931054)]

In [135]:
sorted_beta_gw[-10:]

[(('brace', 489), 0.6413137753615219),
 (('optometrist', 335), 0.6489448572932913),
 (('cold', 165), 0.6706938357652149),
 (('supplement', 230), 0.7421449726185525),
 (('unprofessional', 244), 0.7428650455124668),
 (('adjustment', 217), 0.887529259192033),
 (('she', 21637), 1.0791250711876472),
 (('her', 19092), 1.128201934473994),
 (('herself', 222), 1.2630780907654606),
 (('podiatrist', 160), 1.457689314010105)]

In [136]:
beta_gs = {}
for i,w in enumerate(c[2+S+V-2:2+S+V+S-3]):
    beta_gs[specialties[i]] = w
sorted_beta_gs = sorted(beta_gs.items(), key=operator.itemgetter(1))

In [137]:
sorted_beta_gs

[('brain', -0.7082205747678219),
 ('surg', -0.6107546997064219),
 ('bone', -0.33279325364147433),
 ('ent', -0.27251901014659324),
 ('eye', -0.1879000457090738),
 ('gas', -0.06687276580329325),
 ('psych', -0.0369996537551542),
 ('gyn', 0.08604117023850301),
 ('int', 0.14878041461758204),
 ('endo', 0.20487793356244122),
 ('pedia', 0.3715148475302924),
 ('skin', 0.4108852012842509),
 ('family', 0.5015716176260462)]

In [138]:
beta_sw = {}
start = 2+S+V+S-3 # constant,gender, (skipped) beta_w, beta_s, beta_gw, beta_gs
for i in range(S-1):
    beta_sw_temp = {}
    for j in range(V-1):
        beta_sw_temp[V_new[j]] = c[start + j]
    start += V-1
    beta_sw[specialties[i]] = beta_sw_temp

In [139]:
sorted_beta_sw = {}
for i in specialties[:-1]:
    sorted_beta_sw[i] = sorted(beta_sw[i].items(), key=operator.itemgetter(1))

In [140]:
for i in specialties[:-1]:
    print(" - "+i+":")
    for j in sorted_beta_sw[i][-10:]:
        print(j)
    print('\n')

 - family:
(('acupuncture', 634), 0.7463274339484497)
(('punctual', 323), 0.7581205372665725)
(('est', 618), 0.7848049078926594)
(('listener', 290), 0.7852666132152523)
(('et', 1627), 0.7869973438403535)
(('temp', 172), 0.8018552420976094)
(('en', 471), 0.8493120850825056)
(('tr', 584), 0.8557448116402514)
(('nous', 204), 0.8908003783366911)
(('pour', 589), 0.9277772358281522)


 - surg:
(('surgical', 368), 0.778736059884598)
(('saved', 468), 0.8020704775970456)
(('removed', 599), 0.8319928676018097)
(('operation', 347), 0.9153996476549412)
(('removal', 188), 0.9552687816752649)
(('surgeon', 2102), 0.9903562289025483)
(('lee', 316), 1.0313564087126357)
(('cancer', 626), 1.119877908753179)
(('breast', 750), 1.1215991252604003)
(('orthodontist', 402), 1.7066201170610327)


 - psych:
(('therapy', 660), 0.9700725401749347)
(('drug', 203), 0.9962535415505163)
(('disorder', 177), 1.0082972198431883)
(('anxiety', 597), 1.1825780020623085)
(('session', 609), 1.391069162599734)
(('therapist', 4